In [1]:
!pip install paddlepaddle paddleocr
!pip install opencv-python-headless  # Use opencv-python-headless for Colab to avoid conflicts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.8 MB/s eta 0:00:00:00:01
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=b4332b51aed627b948e1e37fc7aab9b610cf0fccdaa5a81da001c864ac2fd50c
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [2]:
from paddleocr import PaddleOCR
import requests
from PIL import Image
from io import BytesIO

In [3]:
import pandas as pd
import os
import requests
from PIL import Image
from io import BytesIO
from paddleocr import PaddleOCR
import re
class OCRImage:

    def __init__(self, path, ocr_reader, subset=False, size=100):
        self.df = pd.read_csv(path)
        filename_with_ext = os.path.basename(path)
        filename = os.path.splitext(filename_with_ext)[0]
        self.folder_name = "/kaggle/working/Subset_Dataset_Images/" + filename
        self.output_path = "Subset_" + filename + ".csv"
        if not os.path.exists(self.folder_name):
            os.makedirs(self.folder_name)
        if subset:
            self.df = self.df.sample(n=size, random_state=42)
        # Initialize EasyOCR reader
        self.reader = ocr_reader

    # Function to perform OCR on an image from a URL
    def perform_ocr(self, image_url):
        try:
            # Download the image
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))

            # Save the image to the folder
            image_name = os.path.basename(image_url)
            image_path = os.path.join(self.folder_name, image_name)
            img.save(image_path)

            # Perform OCR
            results = self.reader.ocr(image_path)
            results_str = str(results)
            # Use regex to find all measurement values inside parentheses
            cleaned_res = re.findall(r"\('([^']*)'", results_str)
            full_cleaned_res = " ".join(cleaned_res)
            print(full_cleaned_res)
            return cleaned_res,full_cleaned_res, image_path

        except Exception as e:
            return f"Error: {str(e)}", ""

    # Create a new column in the DataFrame for the OCR result and image path
    def getText(self):
        def process_row(row):
            ocr_text,clean_ocr, image_path = self.perform_ocr(row['image_link'])
            return pd.Series([ocr_text,clean_ocr, image_path], index=['ocr_text','clean_ocr','image_path'])

        self.df[['ocr_text','clean_ocr','image_path']] = self.df.apply(process_row, axis=1)

    def getDF(self):
        return self.df

    def saveToCsv(self):
        self.df.to_csv(self.output_path, index=False)

# Example usage:
# ocr = OCRImage('/content/Item_depth.csv')
# ocr.getText()
# ocr.saveToCsv()


In [4]:
# Initialize EasyOCR reader
reader = PaddleOCR(use_angle_cls=True,lang='en')  # Specify the language(s) you want to use


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:17<00:00, 234kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:17<00:00, 582kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 143kiB/s] 

[2024/09/15 09:58:27] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [5]:
Files = ["/kaggle/input/test-set-height-voltage-wattage/height_batch_1.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_2.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_3.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_4.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_5.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_6.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_7.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_8.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_9.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_10.csv"]

In [6]:
Files_1 = [
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_11.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_12.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_13.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_14.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_15.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_16.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_17.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_18.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_19.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_20.csv"]


In [7]:
Files_2 = [
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_21.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_22.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_23.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_24.csv"
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_25.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_26.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_27.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_28.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_29.csv",
    "/kaggle/input/test-set-height-voltage-wattage/height_batch_30.csv"]
    


In [8]:
Files_3 = [
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_31.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_32.csv",
         "/kaggle/input/test-set-height-voltage-wattage/height_batch_33.csv",
         "/kaggle/input/test-set-height-voltage-wattage/voltage_batch_1.csv",
         "/kaggle/input/test-set-height-voltage-wattage/voltage_batch_2.csv",
         "/kaggle/input/test-set-height-voltage-wattage/voltage_batch_3.csv",
         "/kaggle/input/test-set-height-voltage-wattage/voltage_batch_4.csv"]

In [9]:
Files_4 = ["/kaggle/input/test-set-height-voltage-wattage/voltage_batch_5.csv",
         "/kaggle/input/test-set-height-voltage-wattage/voltage_batch_6.csv",
         "/kaggle/input/test-set-height-voltage-wattage/wattage_batch_1.csv",
         "/kaggle/input/test-set-height-voltage-wattage/wattage_batch_2.csv",
         "/kaggle/input/test-set-height-voltage-wattage/wattage_batch_3.csv",
        "/kaggle/input/test-set-height-voltage-wattage/wattage_batch_4.csv",
         "/kaggle/input/test-set-height-voltage-wattage/wattage_batch_5.csv",
         "/kaggle/input/test-set-height-voltage-wattage/wattage_batch_6.csv"]

In [ ]:
%%time
from tqdm.auto import tqdm
import gc

for file in tqdm(Files):#Change the list contents based on the set of files on which OCR has to be performed
    # Step 1: Process the file using the OCR model
    OCR_model = OCRImage(file, reader, False)
    OCR_model.getText()
    OCR_model.saveToCsv()
    
    # Step 2: Delete OCR_model and clear RAM
    del OCR_model
    gc.collect() 

  0%|          | 0/10 [00:00<?, ?it/s]

[2024/09/15 09:58:29] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.617863655090332
[2024/09/15 09:58:30] ppocr DEBUG: cls num  : 3, elapsed : 0.030550241470336914
[2024/09/15 09:58:30] ppocr DEBUG: rec_res num  : 3, elapsed : 0.187896728515625
500mm 120mm 780mm
[2024/09/15 09:58:30] ppocr DEBUG: dt_boxes num : 3, elapsed : 0.3147566318511963
[2024/09/15 09:58:30] ppocr DEBUG: cls num  : 3, elapsed : 0.011353731155395508
[2024/09/15 09:58:31] ppocr DEBUG: rec_res num  : 3, elapsed : 0.39134979248046875
27cm/10.62in 34cm/13.38in 34cm/13.38in
[2024/09/15 09:58:31] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.31596946716308594
[2024/09/15 09:58:31] ppocr DEBUG: cls num  : 4, elapsed : 0.028782129287719727
[2024/09/15 09:58:31] ppocr DEBUG: rec_res num  : 4, elapsed : 0.4443230628967285
45cm 80cm 45cm 1.01.0
[2024/09/15 09:58:32] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.3552241325378418
[2024/09/15 09:58:32] ppocr DEBUG: cls num  : 5, elapsed : 0.034674882888793945
[2024/09/15 09:58:33] ppocr

Zip Folder
